In [3]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.data_preparation import bert_embeddings_from_file
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


e:\Anaconda\envs\LMH\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\lmh23\AppData\Local\Temp\ipykernel_12456\767941787.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# Load data
data = pd.read_csv("./archive/abcnews-date-text.csv")
data = data[:100000]

# Prepare data
unpreprocessed_texts = data['headline_text'].tolist()

vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=2)
vectorizer.fit(unpreprocessed_texts)
processed_texts = vectorizer.transform(unpreprocessed_texts)

words = vectorizer.get_feature_names_out()
processed_texts_list = [' '.join(words[doc.nonzero()[1]]) for doc in processed_texts]


qt = TopicModelDataPreparation("all-mpnet-base-v2")
training_dataset = qt.fit(text_for_contextual=unpreprocessed_texts, text_for_bow=processed_texts_list)

Batches: 100%|██████████| 500/500 [10:09<00:00,  1.22s/it]


In [5]:
# CTM
n_topics = 10
ctm = CombinedTM(bow_size=len(qt.vocab), contextual_size=768, n_components=n_topics, num_epochs=2)

# Train CTM with the prepared dataset
ctm.fit(training_dataset)
ctm_top_words = ctm.get_topics(5)

Epoch: [2/2]	 Seen Samples: [199936/200000]	Train Loss: 48.520204182623594	Time: 0:01:13.399499: : 2it [02:27, 73.97s/it]
100%|██████████| 1563/1563 [00:42<00:00, 36.60it/s]


In [16]:
def print_ctm_top_words(topic_words, n_topics):
    for topic_idx in range(n_topics):
        print("Topic %d:" % topic_idx, end=' ')
        print(' '.join(topic_words[topic_idx]))
        # print('| # topic_count: %d' % ctm.get_topic_count()[topic_idx])

In [17]:
print("CTM Top Words:")
print_ctm_top_words(ctm_top_words, n_topics)

CTM Top Words:
Topic 0: police probe missing search crash
Topic 1: iraq iraqi korea baghdad troops
Topic 2: council water farmers drought mayor
Topic 3: govt funding minister wa indigenous
Topic 4: man charged court charges murder
Topic 5: england test world cup india
Topic 6: qld rail boost coast nsw
Topic 7: wall homework hindering psa wessels
Topic 8: united injury tigers real liverpool
Topic 9: deal trade latham greens pm
